## About Notebook
In this notebook we try to practice all the classification algorithms.

We load a dataset using Pandas library, and apply the following algorithms, and find the best one for this specific dataset by accuracy evaluation methods.

### Required Libraries:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
import pandas as pd
import numpy as np
import matplotlib.ticker as ticker
from sklearn import preprocessing
%matplotlib inline

This dataset is about past loans. The **loan.csv** data set includes details of 346 customers whose loan are already paid off or defaulted. It includes following fields:

| Field          | Description                                                                           |
|----------------|---------------------------------------------------------------------------------------|
| Loan_status    | Whether a loan is paid off on in collection                                           |
| Principal      | Basic principal loan amount at the                                                    |
| Terms          | Origination terms which can be weekly (7 days), biweekly, and monthly payoff schedule |
| Effective_date | When the loan got originated and took effects                                         |
| Due_date       | Since it’s one-time payoff schedule, each loan has one single due date                |
| Age            | Age of applicant                                                                      |
| Education      | Education of applicant                                                                |
| Gender         | The gender of applicant                                                               |

## Load DataFrame form CSV

In [ ]:
df = pd.read_csv('../input/loan-dataset/loan.csv')
df.head()

In [ ]:
df.drop('Unnamed: 0', axis=1, inplace=True)
df.drop('Unnamed: 0.1', axis=1, inplace=True)

In [ ]:
df.head()

### Shape of Data

In [ ]:
df.shape

In [ ]:
df.info()

## Convert to date time object
As the date and time colum are not in proper format, so convert to data time object.


In [ ]:
df['due_date'] = pd.to_datetime(df['due_date'])
df['effective_date'] = pd.to_datetime(df['effective_date'])
df.head()

## Data visualization and pre-processing

Let’s see how many of each class is in our data set

In [ ]:
df['loan_status'].value_counts()

260 people have paid off the loan on time while 86 have gone into collection

Let's plot some columns to underestand data better:

In [ ]:
df.columns

In [ ]:
import seaborn as sns

bins = np.linspace(df.Principal.min(), df.Principal.max(), 7)
g = sns.FacetGrid(df, col="Gender", hue="loan_status", palette="Set1", col_wrap=2)
g.map(plt.hist, 'Principal', bins=bins, ec="k")

g.axes[-1].legend()
plt.show()

In [ ]:
bins = np.linspace(df.age.min(), df.age.max(), 10)
g = sns.FacetGrid(df, col="Gender", hue="loan_status", palette="Set1", col_wrap=2)
g.map(plt.hist, 'age', bins=bins, ec="k")

g.axes[-1].legend()
plt.show()

## Pre-processing: Feature selection/extraction

### Let's look at the day of the week people get the loan

In [ ]:
df['dayofweek'] = df['effective_date'].dt.dayofweek
bins = np.linspace(df.dayofweek.min(), df.dayofweek.max(), 10)
g = sns.FacetGrid(df, col="Gender", hue="loan_status", palette="Set1", col_wrap=2)
g.map(plt.hist, 'dayofweek', bins=bins, ec="k")
g.axes[-1].legend()
plt.show()

We see that people who get the loan at the end of the week don't pay it off, so let's use Feature binarization to set a threshold value less than day 4

In [ ]:
df.columns

In [ ]:
df['week'] = df['dayofweek'].apply(lambda  x:1 if(x>3) else 0)
df.head()

## Convert Categorical features to numerical values

In [ ]:
df.info()

lets look at gender column.


In [ ]:
collection = df.groupby(['Gender'])['loan_status'].value_counts()
collection

86 % of female pay there loans while only 73 % of males pay there loan

### Conversion categorical into numerical
Let's convert male to 0 and female to 1:

In [ ]:
df['Gender'].replace(to_replace=['male','female'], value=[0,1],inplace=True)
# df.drop('Unnamed: 0', axis=1, inplace=True)
# df.drop('Unnamed: 0.1', axis=1, inplace=True)
df.head()

## One Hot Encoding

let's see Education column

In [ ]:
df.groupby(['education'])['loan_status'].value_counts(normalize=True)*100

## Features before One Hot Encoding

In [ ]:
df[['Principal','terms','age','Gender','education']].head()

In [ ]:
features = df[['Principal','terms','age','Gender','week']]
features = pd.concat([features, pd.get_dummies(df['education'])],axis=1)
features.drop(['Master or Above'], axis=1,inplace=True)
features.head()

## Features selection

In [ ]:
X = features
X[0:5]

In [ ]:
y = df['loan_status'].values
y[0:5]

## Normalize Data
Data Standardization give data zero mean and unit variance (technically should be done after train test split)

In [ ]:
X= preprocessing.StandardScaler().fit(X).transform(X)
X[0:5]

## Classification
The following Classification Algorithm will be use here.
1. K Nearest Neighbor(KNN)
2. Decision Tree
3. Support Vector Machine
4. Logistic Regression

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)

In [ ]:
y_pred = knn.predict(X_test)

In [ ]:
print("The Score of the Prediction ",round(knn.score(X_train, y_train)*100 ,2), "%")

## Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
tree = DecisionTreeClassifier(random_state = 25)
tree.fit(X_train, y_train)
y_pred = tree.predict(X_test)
print("The Prediction Score are : ", round(tree.score(X_train, y_train)*100, 2), "%")

In [ ]:
from sklearn  import metrics
print("Train set Data Accuracy :", metrics.accuracy_score(y_train, tree.predict(X_train)))
# print("Test set data Accuracy :", metrics.accuracy_score(y_test, y_pred))
print("Test set Accuracy", metrics.accuracy_score(y_test, tree.predict(X_test)))

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

## Support Vector Machine SVM

In [ ]:
from sklearn import svm

In [ ]:
clf = svm.SVC(kernel='rbf')
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print("The prediction score are : ", round(clf.score(X_train, y_train)*100, 2), "%")

In [ ]:
from sklearn import metrics
print("Train set Accuracy", metrics.accuracy_score(y_train, clf.predict(X_train)))
print("Test set Accuracy", metrics.accuracy_score(y_test, y_pred))

In [ ]:
print(classification_report(y_test, y_pred))

## Performance Comparision

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rm = RandomForestClassifier()
rm.fit(X_train, y_train)
y_pred = rm.predict(X_test)
print("The prediction score are : ", round(rm.score(X_train, y_train)*100, 2), "%")

In [ ]:
from yellowbrick.classifier import ClassificationReport

# K-NearestNeighbour

In [ ]:
visualizer = ClassificationReport(knn, classes=['PAIDOFF','COLLECTION'])
visualizer.fit(X_train, y_train) # Fit the training data to the visualizer
visualizer.score(X_test, y_test) # Evaluate the model on the test data
g = visualizer.poof() # Draw/show/poof the data

## DECISION TREE

In [ ]:
visualizer = ClassificationReport(tree, classes=['PAIDOFF','COLLECTION'])
visualizer.fit(X_train, y_train) # Fit the training data to the visualizer
visualizer.score(X_test, y_test) # Evaluate the model on the test data
g = visualizer.poof() # Draw/show/poof the data

# Support Vector Machine

In [ ]:
visualizer = ClassificationReport(clf, classes=['PAIDOFF','COLLECTION'])
visualizer.fit(X_train, y_train) # Fit the training data to the visualizer
visualizer.score(X_test, y_test) # Evaluate the model on the test data
g = visualizer.poof() # Draw/show/poof the data

# Random Forest 

In [ ]:
visualizer = ClassificationReport(rm, classes=['PAIDOFF','COLLECTION'])
visualizer.fit(X_train, y_train) # Fit the training data to the visualizer
visualizer.score(X_test, y_test) # Evaluate the model on the test data
g = visualizer.poof() # Draw/show/poof the data